In [253]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import io

In [254]:
import mlflow
import mlflow.pytorch

In [255]:
mlflow.set_experiment("MLP_Clasificador_Imagenes")

<Experiment: artifact_location='file:///c:/Users/glouz/Documents/ITBA/4to_Ano/2Q/Redes/skin-dataset-classification-main/mlruns/891879663589540265', creation_time=1764113398632, experiment_id='891879663589540265', last_update_time=1764113398632, lifecycle_stage='active', name='MLP_Clasificador_Imagenes', tags={}>

In [256]:
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils

In [257]:
# Función para loguear una figura matplotlib en TensorBoard
def plot_to_tensorboard(fig, writer, tag, step):
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    image = Image.open(buf).convert("RGB")
    image = np.array(image)
    image = torch.tensor(image).permute(2, 0, 1) / 255.0
    writer.add_image(tag, image, global_step=step)
    plt.close(fig)

In [258]:
# Función para matriz de confusión y clasificación
def log_classification_report(model, loader, writer, step, prefix="val"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    fig_cm, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_dataset.label_encoder.classes_)
    disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
    ax.set_title(f'{prefix.title()} - Confusion Matrix')

    # Guardar localmente y subir a MLflow
    fig_path = f"confusion_matrix_{prefix}_epoch_{step}.png"
    fig_cm.savefig(fig_path)
    mlflow.log_artifact(fig_path)
    os.remove(fig_path)

    plot_to_tensorboard(fig_cm, writer, f"{prefix}/confusion_matrix", step)

    cls_report = classification_report(all_labels, all_preds, target_names=train_dataset.label_encoder.classes_)
    writer.add_text(f"{prefix}/classification_report", f"<pre>{cls_report}</pre>", step)

    # También loguear texto del reporte
    with open(f"classification_report_{prefix}_epoch_{step}.txt", "w") as f:
        f.write(cls_report)
    mlflow.log_artifact(f.name)
    os.remove(f.name)


In [259]:
# Crear directorio de logs
log_dir = "runs/mlp_experimento_1"
log_dir2 = "runs/mlp_experimento_2"
writer = SummaryWriter(log_dir=log_dir)



In [260]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.image_paths = []
        self.labels = []

        class_names = sorted(os.listdir(root_dir))
        self.class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

        for cls in class_names:
            cls_dir = os.path.join(root_dir, cls)
            for fname in os.listdir(cls_dir):
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.image_paths.append(os.path.join(cls_dir, fname))
                    self.labels.append(cls)

        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        label = self.labels[idx]

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented["image"]

        return image, label

In [261]:
train_transform = A.Compose([
    A.Resize(64, 64),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(),
    ToTensorV2()
]) 
"""train_transform = A.Compose([
    A.Resize(64,64),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.Normalize(),
    ToTensorV2()
])"""


'train_transform = A.Compose([\n    A.Resize(64,64),\n    A.HorizontalFlip(p=0.5),\n    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=15, p=0.5),\n    A.RandomBrightnessContrast(p=0.5),\n    A.Normalize(),\n    ToTensorV2()\n])'

In [262]:
val_test_transform = A.Compose([
    A.Resize(64, 64),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(),
    ToTensorV2()
])

In [263]:
# Paths
train_dir = "data/Split_smol/train"
val_dir = "data/Split_smol/val/"

In [264]:
train_dataset = CustomImageDataset(train_dir, transform=train_transform)
val_dataset   = CustomImageDataset(val_dir, transform=val_test_transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)

In [265]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size=64*64*3, dropout=0.0, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight)  # He
                #nn.init.xavier_uniform_(m.weight) #Xavier
                #nn.init.uniform_(m.weight) # Random
                nn.init.zeros_(m.bias)


        

    def forward(self, x):
        return self.model(x)

In [266]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(train_dataset.labels))
model = MLPClassifier(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

In [267]:
# Entrenamiento y validación
def evaluate(model, loader, epoch=None, prefix="val"):
    log_classification_report(model, val_loader, writer, step=epoch, prefix="val")
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss_sum += loss.item()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Loguear imágenes del primer batch
            if i == 0 and epoch is not None:
                img_grid = vutils.make_grid(images[:8].cpu(), normalize=True)
                writer.add_image(f"{prefix}/images", img_grid, global_step=epoch)

    acc = 100.0 * correct / total
    avg_loss = loss_sum / len(loader)

    if epoch is not None:
        writer.add_scalar(f"{prefix}/loss", avg_loss, epoch)
        writer.add_scalar(f"{prefix}/accuracy", acc, epoch)

    return avg_loss, acc

In [268]:
# Loop de entrenamiento
n_epochs = 10
with mlflow.start_run():
    # Log hiperparámetros
    mlflow.log_params({
        "model": "MLPClassifier",
        "input_size": 64*64*3,
        "batch_size": batch_size,
        "lr": 1e-3,
        "epochs": n_epochs,
        "optimizer": "Adam",
        "loss_fn": "CrossEntropyLoss",
        "train_dir": train_dir,
        "val_dir": val_dir,
    })
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
    
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
            images, labels = images.to(device), labels.to(device)
    
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
    
            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
        train_loss = running_loss / len(train_loader)
        train_acc = 100.0 * correct / total
        val_loss, val_acc = evaluate(model, val_loader, epoch=epoch, prefix="val")
    
        print(f"Epoch {epoch+1}:")
        print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
        print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
    
        writer.add_scalar("train/loss", train_loss, epoch)
        writer.add_scalar("train/accuracy", train_acc, epoch)
    
        # Log en MLflow
        mlflow.log_metrics({
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        }, step=epoch)
        # Guardar modelo
        if (epoch == 0):
            print("test")
            for name, param in model.named_parameters():
                writer.add_histogram(f"weights/{name}", param, epoch)
                if param.grad is not None:
                    writer.add_histogram(f"grads/{name}", param.grad, epoch)

    torch.save(model.state_dict(), "mlp_model.pth")
    print("Modelo guardado como 'mlp_model.pth'")
    mlflow.log_artifact("mlp_model.pth")
    mlflow.pytorch.log_model(model, artifact_path="pytorch_model")
    print("Modelo guardado como 'mlp_model.pth'")

Epoch 1/10: 100%|██████████| 22/22 [00:07<00:00,  2.81it/s]
c:\Users\glouz\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\glouz\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\glouz\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

Epoch 1:
  Train Loss: 13.9517, Accuracy: 22.99%
  Val   Loss: 7.3461, Accuracy: 21.67%
test


Epoch 2/10: 100%|██████████| 22/22 [00:07<00:00,  3.03it/s]


Epoch 2:
  Train Loss: 4.3938, Accuracy: 36.49%
  Val   Loss: 4.5054, Accuracy: 30.56%


Epoch 3/10: 100%|██████████| 22/22 [00:07<00:00,  3.01it/s]
c:\Users\glouz\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\glouz\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\glouz\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division`

...

Epoch 9:
  Train Loss: 1.3874, Accuracy: 59.91%
  Val   Loss: 2.3789, Accuracy: 53.89%


Epoch 10/10: 100%|██████████| 22/22 [00:07<00:00,  2.88it/s]


Epoch 10:
  Train Loss: 1.1777, Accuracy: 63.79%
  Val   Loss: 2.4985, Accuracy: 51.67%
Modelo guardado como 'mlp_model.pth'


2025/11/25 22:50:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'mlp_model.pth'


Registro de resultados antes de cada cambio:
1) Sin cambios: Train Loss: 0.9646, Accuracy: 62.36% // Val   Loss: 1.3554, Accuracy: 54.44%
2) Dropout:  
a) p=0.5 Train Loss: 1.6273, Accuracy: 39.08% // Val   Loss: 1.3873, Accuracy: 47.22%
b) p=0.1 Train Loss: 1.1150, Accuracy: 57.04%  // Val   Loss: 1.1835, Accuracy: 54.44%
Es lógico que baje la precisión en entrenamiento, pero no se ven mejoras en validación porque el modelo original no sufría overfitting.
3) BatchNorm: Train Loss: 1.1000, Accuracy: 56.75% // Val   Loss: 1.0953, Accuracy: 56.67%
4) L2 (Con BatchNorm): Train Loss: 1.1527, Accuracy: 53.45% // Val   Loss: 1.0540, Accuracy: 57.78%
5) Data Augmentation:  Train Loss: 1.3754, Accuracy: 44.25% // Val   Loss: 1.1647, Accuracy: 48.89%


Registro de cambios con inicialización de parámetros:

1) He: Train Loss: 1.1906, Accuracy: 64.80% // Val   Loss: 1.9943, Accuracy: 50.56%
2) Xavier: Train Loss: 1.1421, Accuracy: 61.93% // Val   Loss: 1.6970, Accuracy: 48.89%
3) Random: Train Loss: 66447.2658, Accuracy: 17.53% // Val   Loss: 63117.0649, Accuracy: 18.89%

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement imghdr (from versions: none)

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for imghdr


In [272]:
!tensorboard --logdir=runs/mlp_experimento_1

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\glouz\AppData\Local\Programs\Python\Python313\Scripts\tensorboard.exe\__main__.py", line 4, in <module>
    from tensorboard.main import run_main
  File "C:\Users\glouz\AppData\Local\Programs\Python\Python313\Lib\site-packages\tensorboard\main.py", line 27, in <module>
    from tensorboard import default
  File "C:\Users\glouz\AppData\Local\Programs\Python\Python313\Lib\site-packages\tensorboard\default.py", line 40, in <module>
    from tensorboard.plugins.image import images_plugin
  File "C:\Users\glouz\AppData\Local\Programs\Python\Python313\Lib\site-packages\tensorboard\plugins\image\images_plugin.py", line 18, in <module>
    import imghdr
ModuleNotFoundError: No module named 'imghdr'
